In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import  train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

In [4]:
df=messages=pd.read_csv('/content/SMSSpamCollection.txt',sep='\t',names=['label','message'])
df.head(5)

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [6]:
df.shape

(5572, 2)

In [7]:
 x=list(df['message'])

In [8]:
y=list(df['label'])

In [9]:
y=list(pd.get_dummies(y,drop_first=True)['spam'])
from sklearn .model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=0)

In [10]:
!pip install transformers


In [11]:
from transformers import DistilBertTokenizerFast
tokenizer=DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [12]:
train_encodings=tokenizer(x_train,truncation=True,padding=True)
test_encodings=tokenizer(x_test,truncation=True,padding=True)

In [13]:
import tensorflow as tf
train_dataset=tf.data.Dataset.from_tensor_slices((dict(train_encodings),y_train))
test_dataset=tf.data.Dataset.from_tensor_slices((dict(test_encodings),y_test))

In [14]:
# import torch
# from torch.utils.data import TensorDataset, DataLoader

# # Assuming train_encodings, test_encodings, y_train, y_test are already defined

# # Convert data to PyTorch tensors
# train_encodings_pt = {key: torch.tensor(val) for key, val in train_encodings.items()}
# test_encodings_pt = {key: torch.tensor(val) for key, val in test_encodings.items()}
# y_train_pt = torch.tensor(y_train)
# y_test_pt = torch.tensor(y_test)

# # Create TensorDataset objects
# train_dataset = TensorDataset(train_encodings_pt['input_ids'], train_encodings_pt['attention_mask'], y_train_pt)
# test_dataset = TensorDataset(test_encodings_pt['input_ids'], test_encodings_pt['attention_mask'], y_test_pt)

# # Create DataLoader objects for batching and shuffling (optional)
# train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)  # Adjust batch_size as needed
# test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [15]:
train_dataset


<_TensorSliceDataset element_spec=({'input_ids': TensorSpec(shape=(238,), dtype=tf.int32, name=None), 'attention_mask': TensorSpec(shape=(238,), dtype=tf.int32, name=None)}, TensorSpec(shape=(), dtype=tf.bool, name=None))>

In [17]:
!pip install tensorflow

In [18]:
!pip install transformers

  Using cached TFTrainer-0.2.2-py3-none-any.whl.metadata (588 bytes)
ERROR: Could not find a version that satisfies the requirement from (from versions: none)
ERROR: No matching distribution found for from


In [26]:
from transformers import TFDistilBertForSequenceClassification,  TFTrainingArguments,Trainer
training_arg = TFTrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=2,              # total number of training epochs
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=16,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)

In [39]:
with training_arg.strategy.scope():
    model = TFDistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")
    trainer = Trainer(
    model=model,
    args=training_arg,
    train_dataset=train_dataset,eval_dataset=test_dataset

)
trainer.train()

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_projector.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

AttributeError: 'TFDistilBertForSequenceClassification' object has no attribute 'to'

In [29]:
Trainer= Trainer(model=model,args=training_arg,train_dataset=train_dataset,eval_dataset=test_dataset)
Trainer.train()

AttributeError: 'TFDistilBertForSequenceClassification' object has no attribute 'to'